# Clasificacion de Categorias usando TF-IDF

Autor: Remberto López

# Problema

Se busca clasificar automáticamente textos en tres categorías diferentes: deportivos, políticos y de entretenimiento. Se tiene un corpus de textos divididos en estas tres categorías para entrenar el modelo. Se desarrollará un sistema de clasificación que pueda determinar a qué categoría pertenece un nuevo texto de consulta.

El corpus de entrenamiento consta de una colección de textos deportivos, políticos y de entretenimiento. Cada texto está representado como un documento, y se tiene una lista de estos documentos junto con sus etiquetas de categoría correspondientes.

Se utilizará la técnica de codificación TF-IDF (Term Frequency-Inverse Document Frequency) para convertir los documentos de texto en vectores numéricos, lo que nos permitirá cuantificar la importancia relativa de cada palabra en cada documento. Luego, se calculará la similitud del coseno entre el texto de consulta  (query) y cada una de las categorías para determinar que categoría es más probable a la que pertenezca.

Los objetivos son:

1. Preprocesamiento de los textos: carga de textos, limpieza de texto y eliminación de stop words.
2. Crear un modelo de codificación TF-IDF.
3. Implementar la similitud del coseno entre el texto de consulta y cada categoría.
4. Desarrollar una función de clasificación que tome un texto de consulta como entrada y devuelva la categoría más probable.

# 1. Procesamiento de datos

## 1.1 Carga de datos

In [ ]:
import math
import glob
import os

PATH = "corpus/"
contenido = glob.glob(PATH+"*.txt")
diccionario_corpus = {}
for textos in contenido:
  with open(textos,"r") as textos_txt:
    corpus = textos_txt.read()
  categoria = os.path.basename(textos)[:-6]
  if categoria in diccionario_corpus.keys():
    diccionario_corpus[categoria].append(corpus)
  else:
    diccionario_corpus[categoria] = [corpus]

In [ ]:
for categoria,corpus in diccionario_corpus.items(): #Usamos este metodo .items() para iterar segun categorias y sus respectivos corpus para ver si todo esta bien guardado
  print(f"{categoria} : {corpus}")

deporte : ['El equipo de fútbol local logró una emocionante victoria en las semifinales del campeonato regional anoche. Con un gol en el tiempo de descuento, aseguraron su lugar en la final contra su rival histórico. Los aficionados están eufóricos y se espera un lleno total en el estadio para el enfrentamiento decisivo el próximo fin de semana.\n\n\n', 'La estrella del tenis, que ha dominado las canchas durante la última década, sorprendió al mundo del deporte al anunciar su retiro del circuito profesional. Con múltiples títulos de Grand Slam y una carrera brillante, deja un legado imborrable en el tenis mundial. Los fanáticos lamentan su partida, pero celebran sus innumerables logros y momentos inolvidables en el deporte.\n\n\n', 'El equipo de baloncesto de la NBA ha cerrado un acuerdo histórico al firmar a una joven promesa del deporte por un contrato millonario. Con habilidades excepcionales y un potencial ilimitado, esta nueva adquisición promete llevar al equipo a nuevas alturas 

## 1.2 Limpieza de texto
### 1.2.1 Limpieza de simbolos

In [ ]:
def limpieza_simbolos(values, simbolos):
    """
    Elimina un conjunto de símbolos de una lista de textos y convierte el texto resultante a minúsculas.

    Args:
        values (list of str): Lista de textos en los cuales se eliminarán los símbolos.
        simbolos (list of str): Lista de símbolos que se deben eliminar de los textos.

    Returns:
        list of str: Lista de textos procesados, sin los símbolos indicados y en minúsculas.
    """
    lista_values_limpios = []
    for corpus in values:
        texto = corpus
        for simbolo in simbolos:
            texto = texto.replace(simbolo, "")
        lista_values_limpios.append(texto.lower())  # Aprovechamos de dejar todo en minúsculas con .lower()
    return lista_values_limpios


In [ ]:
simbolos_redundantes = [ ".",
    ",",  # Comas
    ";",  # Puntos y comas
    "()", "[]", "{}",  # Paréntesis redundantes
    "'", '"',  # Comillas redundantes
    "  ", "\t", "\n",  # Espacios en blanco excesivos
    "+", "-", "*", "/",  # Operadores redundantes
]
for categoria, values in diccionario_corpus.items(): #iteramos en funcion de la categoria y sus respectivos valores
  diccionario_corpus[categoria] = limpieza_simbolos(values,simbolos_redundantes) # el texto de los values(corpus) se modifican y se agregan directamente en su respectiva categoria

## 1.2.2 Limpieza de Stop-Words

In [ ]:
def limpieza_stop_words(values, stop_words):
    """
    Elimina las stop words de una lista de textos.

    Args:
        values (list of str): Lista de textos que se desea procesar.
        stop_words (list of str): Lista de palabras consideradas stop words que se deben eliminar de los textos.

    Returns:
        list of str: Lista de textos procesados, sin las stop words.
    """
    lista_values_limpios = []  # Se inicializa una lista vacía para almacenar los textos después de limpiar las stop words.
    for corpus in values:  # Iteramos sobre cada texto en la lista de valores de entrada.
        texto = corpus  # Tomamos el texto actual para procesar.
        palabras = texto.split()  # Dividimos el texto en palabras individuales.
        palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]  # Filtramos las palabras eliminando las que se encuentren en la lista de stop words.
        texto_filtrado = " ".join(palabras_filtradas)  # Unimos las palabras filtradas de vuelta en un solo texto.
        lista_values_limpios.append(texto_filtrado)  # Añadimos el texto filtrado a la lista de textos limpios.

    return lista_values_limpios


In [ ]:
# Palabras redudantes en español
stop_words_espanol = [
    "a", "al", "algo", "algunas", "algunos", "ante", "antes",
    "como", "con", "contra", "cual", "cuando", "de", "del",
    "desde", "donde", "durante", "e", "el", "ella", "ellas",
    "ellos", "en", "entre", "era", "erais", "eran", "eras",
    "eres", "es", "esa", "esas", "ese", "eso", "esos", "esta",
    "estaba", "estabais", "estaban", "estabas", "estad", "estada",
    "estadas", "estado", "estados", "estamos", "estando", "estar",
    "estaremos", "estará", "estarán", "estarás", "estaré", "estaréis",
    "estaría", "estaríais", "estaríamos", "estarían", "estarías",
    "estas", "este", "estemos", "esto", "estos", "estoy", "estuve",
    "estuviera", "estuvierais", "estuvieran", "estuvieras", "estuvieron",
    "estuviese", "estuvieseis", "estuviesen", "estuvieses", "estuvimos",
    "estuviste", "estuvisteis", "estuviéramos", "estuviésemos", "estuvo",
    "está", "estábamos", "estáis", "están", "estás", "esté", "estéis",
    "estén", "estés", "fue", "fuera", "fuerais", "fueran", "fueras",
    "fueron", "fuese", "fueseis", "fuesen", "fueses", "fui", "fuimos",
    "fuiste", "fuisteis", "fuéramos", "fuésemos", "ha", "habida",
    "habidas", "habido", "habidos", "habiendo", "habremos", "habrá",
    "habrán", "habrás", "habré", "habréis", "habría", "habríais",
    "habríamos", "habrían", "habrías", "habéis", "había", "habíais",
    "habíamos", "habían", "habías", "han", "has", "hasta", "hay",
    "haya", "hayamos", "hayan", "hayas", "hayáis", "he", "hemos",
    "hube", "hubiera", "hubierais", "hubieran", "hubieras", "hubieron",
    "hubiese", "hubieseis", "hubiesen", "hubieses", "hubimos", "hubiste",
    "hubisteis", "hubiéramos", "hubiésemos", "hubo", "la", "las", "le",
    "les", "lo", "los", "me", "mi", "mis", "mucho", "muchos", "muy",
    "más", "mí", "mía", "mías", "mío", "míos", "nada", "ni", "no",
    "nos", "nosotras", "nosotros", "nuestra", "nuestras", "nuestro",
    "nuestros", "o", "os", "otra", "otras", "otro", "otros", "para",
    "pero", "poco", "por", "porque", "que", "quien", "quienes", "qué",
    "se", "sea", "seamos", "sean", "seas", "sentid", "sentida", "sentidas",
    "sentido", "sentidos", "seremos", "será", "serán", "serás", "seré",
    "seréis", "sería", "seríais", "seríamos", "serían", "serías", "seáis",
    "siente", "sin", "sintiendo", "sobre", "sois", "somos", "son", "soy",
    "su", "sus", "suya", "suyas", "suyo", "suyos", "sí", "también", "tanto",
    "te", "tendremos", "tendrá", "tendrán", "tendrás", "tendré", "tendréis",
    "tendría", "tendríais", "tendríamos", "tendrían", "tendrías", "tened",
    "tenemos", "tenga", "tengamos", "tengan", "tengas", "tengo", "tengáis",
    "tenida", "tenidas", "tenido", "tenidos", "teniendo", "tenéis", "tenía",
    "teníais", "teníamos", "tenían", "tenías", "ti", "tiene", "tienen", "toda",
    "todas", "todo", "todos", "tu", "tus", "tuve", "tuviera", "tuvierais",
    "tuvieran", "tuvieras", "tuvieron", "tuviese", "tuvieseis", "tuviesen",
    "tuvieses", "tuvimos", "tuviste", "tuvisteis", "tuviéramos", "tuviésemos",
    "tuvo", "tuya", "tuyas", "tuyo", "tuyos", "tú", "un", "una", "uno", "unos",
    "vosotras", "vosotros", "vuestra", "vuestras", "vuestro", "vuestros", "y",
    "ya", "yo"
]
for categoria,values in diccionario_corpus.items(): #se usa misma lógica que en limpieza de simbolos, es decir, retorno del corpus ya limpio y que se cambia por el antiguo en su respectiva categoria
  diccionario_corpus[categoria] = limpieza_stop_words(values, stop_words_espanol)

# 2. Codificación TF-IDF

In [ ]:
def calculo_tf(texto, palabra_diccionario):
    """
    Calcula la frecuencia de término (TF) de una palabra en un texto.

    Args:
        texto (str): Texto en el que se desea calcular la frecuencia de la palabra.
        palabra_diccionario (str): Palabra cuya frecuencia se desea calcular en el texto.

    Returns:
        float: Frecuencia de término de la palabra en el texto, calculada como el número 
            de apariciones de la palabra dividido entre el total de palabras en el texto.
    """
    # tf = repetición de la palabra en el documento / total de palabras del documento
    total_palabras = len(texto.split())
    repeticion_palabra = texto.count(palabra_diccionario)
    tf = repeticion_palabra / total_palabras
    return tf


In [ ]:
def calculo_idf(diccionario_corpus, palabra_diccionario):
    """
    Calcula la frecuencia inversa de documentos (IDF) de una palabra en un corpus.

    Args:
        diccionario_corpus (dict): Diccionario donde las claves representan categorías y los valores 
            son listas de textos (documentos). Se usa para calcular cuántos documentos contienen la palabra.
        palabra_diccionario (str): Palabra cuya IDF se desea calcular.

    Returns:
        float: Valor de la IDF para la palabra dada, calculado como el logaritmo base 10 del número de documentos 
            con la palabra dividido entre el número total de documentos en el corpus. Si la palabra no aparece en 
            ningún documento, se retorna 1 para evitar un valor de IDF indefinido.
    """
    # idf = log(numero de docs con la palabra / numero total de docs)
    numero_docs_con_palabra = 0
    total_docs = 0
    for categoria in diccionario_corpus.keys():
        for corpus in diccionario_corpus[categoria]:
            texto = corpus
            palabras = texto.split()
            if palabra_diccionario in palabras:  # Si la palabra del diccionario está en el texto
                numero_docs_con_palabra += 1  # Aumentamos los documentos con la palabra
            total_docs += 1  # Contamos el total de documentos

    if numero_docs_con_palabra == 0:  # log(0) no existe, por lo tanto, asignamos idf = 1
        idf = 1
    else:
        idf = math.log10(numero_docs_con_palabra / total_docs)  # Calculamos IDF
    return idf



In [ ]:
def codificacion_tf_idf(values, diccionario, diccionario_corpus):
    """
    Calcula la codificación TF-IDF para una lista de textos, donde cada texto se representa 
    mediante un vector de características basado en el TF-IDF de las palabras de un diccionario.

    Args:
        values (list of str): Lista de textos (corpus) a codificar.
        diccionario (list of str): Lista de palabras que conforman el vocabulario para calcular 
            el TF-IDF.
        diccionario_corpus (dict): Diccionario que representa el corpus completo, donde las claves son 
            categorías y los valores son listas de textos (documentos). Se utiliza para calcular el IDF.

    Returns:
        list of list of float: Lista de vectores, donde cada sublista representa la codificación TF-IDF 
            de un texto, calculada para cada palabra del diccionario.
    """
    # Cálculo de tf-idf = (repetición de palabra en un documento / palabras totales del documento) 
    # / log10(docs con esa palabra / total de documentos)
    lista_codificacion = []
    for corpus in values:
        texto = corpus
        # Iteramos sobre las palabras del diccionario para calcular el vector de codificación TF-IDF
        vector = [calculo_tf(texto, palabra_diccionario.lower()) / calculo_idf(diccionario_corpus, palabra_diccionario.lower()) for palabra_diccionario in diccionario]
        lista_codificacion.append(vector)  # Añadimos el vector de codificación a la lista
        vector = []  # Limpiamos el vector para la próxima iteración
    return lista_codificacion


In [ ]:
# Diccionario de palabras para la codificación
diccionario = ("Equipo", "Final", "Victoria", "Retiro", "Estrella", "Contrato",
               "Baloncesto", "Lesión", "Campeón", "Competencia", "Presidente",
               "Corrupción", "Escándalo", "Conflictos", "Legislación", "Partidos",
               "Reforma", "Oposición", "Campaña", "Democracia", "Película", "Estreno",
               "Pop", "Álbum", "Festival", "Cine", "Estrella", "Canciones", "Actuaciones",
               "Celebración")

In [ ]:
diccionario_corpus_codificacion = {} #creamos un nuevo diccionario donde se guardaran los corpus codificados (con mismas claves)
for categoria,values in diccionario_corpus.items():
  if categoria in diccionario_corpus_codificacion.keys():
    diccionario_corpus_codificacion.append(codificacion_tf_idf(values,diccionario,diccionario_corpus))
  else:
    diccionario_corpus_codificacion[categoria] = codificacion_tf_idf(values,diccionario,diccionario_corpus) #en cada categoria se agregara al codificacion de su respectivo corpus

In [ ]:
for categoria,corpus in diccionario_corpus_codificacion.items(): #Usamos este metodo .items() para iterar segun categorias y sus respectivos corpus para ver si todo esta bien guardado
  print(f"{categoria} : {corpus}")

deporte : [[-0.057274622325644176, -0.0639054074502846, -0.0319527037251423, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0], [-0.0, -0.0, -0.0, -0.03894233966480209, -0.03894233966480209, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.03894233966480209, -0.0, -0.0, 0.0], [-0.16609640474436813, -0.0, -0.0, -0.0, -0.0, -0.030887613600970892, -0.030887613600970892, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0], [-0.05357948540140907, -0.0, -0.0, -0.04145474867543447, -0.0, -0.0, -0.0, -0.059782477937363014, -0.029891238968681507, -0.029891238968681507, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0]]
politica : [[-0.0, -0.0, -0.0, -0.0, -0.0, -0

# 3. Calculo similitudes
## 3.1 Similitud del Coseno

In [ ]:
def coseno(query_codificado, corpus_codificado):
    """
    Calcula la similitud coseno entre dos vectores codificados.

    La similitud coseno se calcula como el producto escalar de los vectores dividido entre el producto de 
    sus longitudes (módulos). La fórmula es: similitud_coseno = (a * b) / (|a| * |b|), donde:
        - a es el vector codificado del corpus.
        - b es el vector codificado de la consulta (query).
        - |a| es la magnitud del vector a, calculada como la raíz cuadrada de la sumatoria de los cuadrados de sus elementos.
        - |b| es la magnitud del vector b, calculada de la misma manera.

    Args:
        query_codificado (list of float): Vector codificado que representa la consulta (query).
        corpus_codificado (list of float): Vector codificado que representa un documento del corpus.

    Returns:
        float: Valor de la similitud coseno entre los dos vectores. Si alguno de los vectores tiene magnitud cero, 
            se retorna 0.
    """
    sumatoria_a_cuadrado = 0
    sumatoria_b_cuadrado = 0
    sumatoria_a_por_b = 0

    # Calculamos la sumatoria de los cuadrados de los elementos de ambos vectores
    for a in corpus_codificado:
        sumatoria_a_cuadrado += float(a) ** 2

    for b in query_codificado:
        sumatoria_b_cuadrado += float(b) ** 2

    # Calculamos el producto escalar entre los dos vectores
    for a, b in zip(corpus_codificado, query_codificado):
        sumatoria_a_por_b += float(a) * float(b)

    # Si la magnitud de alguno de los vectores es 0, la similitud coseno es 0
    if sumatoria_a_cuadrado == 0 or sumatoria_b_cuadrado == 0:
        similitud_coseno = 0
    else:
        similitud_coseno = sumatoria_a_por_b / (math.sqrt(sumatoria_a_cuadrado) * math.sqrt(sumatoria_b_cuadrado))

    return similitud_coseno


## 3.2 Similitud Euclidea

In [ ]:
def euclides(query_codificado, corpus_codificado):
    """
    Calcula la distancia euclidiana entre dos vectores codificados.

    La distancia euclidiana se calcula utilizando la fórmula:
        distancia_euclidea = sqrt(sumatoria((b - a)^2)), donde:
            - a es el vector codificado de la consulta (query).
            - b es el vector codificado del documento del corpus.

    Args:
        query_codificado (list of float): Vector codificado que representa la consulta (query).
        corpus_codificado (list of float): Vector codificado que representa un documento del corpus.

    Returns:
        float: Distancia euclidiana entre los dos vectores. Un valor más bajo indica mayor similitud.
    """
    sumatoria_b_menos_a_cuadrado = 0
    for a, b in zip(corpus_codificado, query_codificado):
        sumatoria_b_menos_a_cuadrado += ((float(b) - float(a)) ** 2)
    euclides = math.sqrt(sumatoria_b_menos_a_cuadrado)
    return euclides


## 3.3 Similitud

In [ ]:
def similitud(query_codificado, diccionario_corpus_codificacion, num):
    """
    Calcula la categoría más similar a una consulta (query) codificada utilizando una métrica de similitud.

    La función utiliza dos métricas de similitud para comparar un vector de consulta codificado con los vectores 
    codificados en un corpus: 
    - Similitud del coseno (cuando `num` es 1).
    - Distancia euclidiana (cuando `num` es 2).

    Args:
        query_codificado (list of float): Vector codificado que representa la consulta (query).
        diccionario_corpus_codificacion (dict): Diccionario donde las claves son categorías y los valores son 
            listas de vectores codificados que representan los documentos de cada categoría.
        num (int): Parámetro que indica qué métrica de similitud usar:
            - 1 para similitud coseno.
            - 2 para distancia euclidiana.

    Returns:
        str: La categoría del vector más similar al query, basado en la métrica de similitud especificada.
            Para la similitud coseno, se retorna la categoría con el valor más alto de similitud.
            Para la distancia euclidiana, se retorna la categoría con el valor más bajo de distancia.

    Notes:
        - Si `num` es 1, la función busca el vector con la mayor similitud coseno.
        - Si `num` es 2, la función busca el vector con la menor distancia euclidiana.
    """
    # Inicialización de la variable que almacenará la mayor similitud encontrada.
    if num == 1:
        mayor_similitud = -1000  # Valor inicial bajo para asegurar que cualquier similitud real será mayor.
    elif num == 2:
        mayor_similitud = float('inf')  # Valor inicial alto para asegurar que cualquier distancia real será menor.

    # Variable para almacenar la categoría del vector más similar.
    categoria_query = ""

    # Iteramos sobre cada categoría y sus vectores codificados en el diccionario.
    for categoria, corpus_codificados in diccionario_corpus_codificacion.items():
        # Iteramos sobre cada vector dentro de una categoría específica.
        for vector in corpus_codificados:
            # Calculamos la similitud del coseno si num es 1.
            if num == 1:
                similitud_actual = coseno(query_codificado, vector)
            # Calculamos la distancia euclídea si num es 2.
            elif num == 2:
                similitud_actual = euclides(query_codificado, vector)

            # Actualizamos la mayor similitud y la categoría correspondiente.
            # Para la similitud del coseno, buscamos valores más altos.
            # Para la distancia euclídea, buscamos valores más bajos.
            if (num == 1 and similitud_actual > mayor_similitud) or (num == 2 and similitud_actual < mayor_similitud):
                mayor_similitud = similitud_actual
                categoria_query = categoria

    # Devolvemos la categoría del vector más similar.
    return categoria_query


# 4. Clasificar
# 4.1 Carga de querys

In [ ]:
PATH = "querys/"
contenido = glob.glob(PATH+"*.txt")
diccionario_querys = {}  # Inicializa como una lista vacía
for clave,archivo in enumerate(contenido):
    with open(archivo, "r") as query_txt:
        query = query_txt.read()
    if clave in diccionario_querys.keys():
      diccionario_querys[clave].append(query)
    else:
      diccionario_querys[clave] = query #en este caso cada query tendra una clave (llave) unica, ya que no sabemos a categoria estan asociados

## 4.1.1 Limpiamos Querys

In [ ]:
def limpieza_query(query, simbolos, stop_words):
    """
    Limpia una consulta (query) eliminando símbolos y palabras de parada (stop words).

    La función realiza dos pasos de limpieza en la consulta:
    1. Elimina los símbolos no deseados de la consulta utilizando una lista de símbolos proporcionada.
    2. Elimina las palabras de parada (stop words) de la consulta, ignorando mayúsculas y minúsculas.

    Args:
        query (str): Texto de la consulta a limpiar.
        simbolos (list of str): Lista de símbolos que deben eliminarse del texto.
        stop_words (list of str): Lista de palabras de parada que deben eliminarse del texto.

    Returns:
        str: La consulta limpia, con los símbolos y las palabras de parada eliminadas, y las palabras en minúsculas.

    Notes:
        - Los símbolos se reemplazan por un espacio vacío.
        - Las palabras se convierten a minúsculas antes de comparar con las stop words.
    """
    texto = query
    # Eliminamos los símbolos no deseados
    for simbolo in simbolos:
        texto = texto.replace(simbolo, "")
    # Filtramos las palabras que no están en stop words
    palabras = texto.split()
    palabras_filtradas = [palabra.lower() for palabra in palabras if palabra.lower() not in stop_words]
    # Unimos las palabras filtradas para obtener el texto limpio
    texto_filtrado = " ".join(palabras_filtradas)
    return texto_filtrado


In [ ]:
for clave, query in diccionario_querys.items():
  diccionario_querys[clave] = limpieza_query(query, simbolos_redundantes, stop_words_espanol)

In [ ]:
for clave, valor in diccionario_querys.items():
    print(f"Clave: {clave}, Valor: {valor}")

Clave: 0, Valor: equipo nacional fútbol logra histórica victoria final campeonato mundial asegurando trofeo codiciado deporte después enfrentar rivales formidables valentía determinación llevaron triunfo generando gran orgullo celebración país victoria resultado años trabajo dedicación equipo espera inspirar futuras generaciones deportistas
Clave: 1, Valor: nuevo gobierno electo presenta agenda legislativa próximo año destacando prioridades crecimiento económico empleo salud educación medio ambiente espera diálogo constructivo partidos políticos abordar desafíos nacionales objetivo mejorar calidad vida ciudadanos promover cambio progreso país
Clave: 2, Valor: estreno mundial esperada película ciencia ficción orígenes: allá universo dirigida alex west evento lleno anticipación emoción película elogiada efectos visuales actuaciones promete llevar espectadores experiencia cinematográfica precedentes explorando misterios cosmos desafiando límites imaginación
Clave: 3, Valor: nueva película

## 4.2 Codificacion de querys

In [ ]:
def calculo_idf_query(diccionario_querys, palabra_diccionario):
    """
    Calcula el valor de IDF (Inverse Document Frequency) para una palabra específica dentro de un conjunto de consultas.

    El IDF se calcula usando la siguiente fórmula:
    IDF = log10(numero de consultas que contienen la palabra / número total de consultas).
    
    Si una palabra no aparece en ninguna consulta, el valor de IDF es 1.
    Si el valor de IDF es cero debido a la rareza de la palabra, también se ajusta a 1 para evitar errores en el cálculo posterior.

    Args:
        diccionario_querys (dict): Diccionario donde las claves son identificadores de consultas y los valores son las consultas (strings) completas.
        palabra_diccionario (str): Palabra para la cual se desea calcular el valor IDF.

    Returns:
        float: Valor de IDF para la palabra dada, que mide su importancia inversa en el conjunto de consultas.

    Notes:
        - Si la palabra no aparece en ninguna consulta, el IDF es 1 por defecto.
        - Si el valor de IDF calculado es 0, también se ajusta a 1 para evitar divisiones por cero o problemas con valores muy pequeños.
    """
    numero_docs_con_palabra = 0
    total_docs = 0

    for clave in diccionario_querys.keys():
        texto = diccionario_querys[clave]
        palabras = texto.split()
        if palabra_diccionario in palabras:
            numero_docs_con_palabra += 1
        total_docs += 1

    if numero_docs_con_palabra == 0:
        idf = 1
    else:
        idf = math.log10(numero_docs_con_palabra / total_docs)
        if idf == 0:  # Prevención de IDF igual a 0 para evitar problemas de cálculo posteriores
            idf = 1

    return idf


In [ ]:
def codificacion_tf_idf_querys(query, diccionario, diccionario_querys):
    """
    Calcula la codificación TF-IDF para una consulta (query) usando un diccionario de términos y un diccionario de consultas.

    La función calcula el vector TF-IDF para una consulta específica, donde:
    - TF (Term Frequency) se calcula como la frecuencia de una palabra en el documento (consulta) dividida entre el total de palabras en el documento.
    - IDF (Inverse Document Frequency) se calcula utilizando el diccionario de todas las consultas, en lugar de un corpus completo.

    Args:
        query (str): Texto de la consulta que se desea codificar.
        diccionario (list of str): Lista de palabras del diccionario que se utilizarán para la codificación.
        diccionario_querys (dict): Diccionario que contiene las consultas previas y su respectiva codificación, 
            utilizado para calcular el IDF de cada palabra.

    Returns:
        list of float: Vector de codificación TF-IDF para la consulta dada, donde cada valor en el vector 
            representa el peso TF-IDF de una palabra del diccionario en la consulta.

    Notes:
        - La función asume que las palabras en el diccionario están en minúsculas para evitar discrepancias por mayúsculas.
        - Para cada palabra en el diccionario, se calcula su TF y IDF, y luego se realiza la codificación TF-IDF.
    """
    texto = query
    vector = [calculo_tf(texto, palabra_diccionario.lower()) / calculo_idf_query(diccionario_querys, palabra_diccionario.lower()) for palabra_diccionario in diccionario]
    return vector


In [ ]:
diccionario_querys_codificacion = {}
for clave,query in diccionario_querys.items():
  if clave in diccionario_corpus_codificacion.keys():
    pass #Como clave es unica sabemos que nunca entrara aqui por lo cual es omitible
  else:
    diccionario_querys_codificacion[clave] = codificacion_tf_idf_querys(query,diccionario, diccionario_querys)


In [ ]:
for clave in diccionario_querys_codificacion.keys(): #Comprobacion de todo en orden
  print(diccionario_querys_codificacion[clave])

[-0.08978184040236115, -0.04489092020118057, -0.05985456026824077, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.029927280134120385]
[-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.06904676911140561, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.10066448772385947, -0.03355482924128649, 0.0, 0.0, -0.0, 0.030303030303030304, 0.0, 0.0, -0.03355482924128649, -0.0]
[-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.1277664651879755, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0]
[-0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, -0.04814388543315018, 0.0, 0.0, 0.0, -0.0, -0.0]
[-0.15818705213749

## 4.3 Clasificar

In [ ]:
def clasificar(query_codificado, diccionario_corpus_codificacion, num):
    """
    Clasifica una consulta codificada (query) en la categoría más similar dentro de un diccionario de corpus codificados.

    La función utiliza una medida de similitud (coseno o euclídea) para comparar el vector codificado de la consulta con los vectores codificados en el diccionario.
    Dependiendo del valor de `num`, se utiliza la similitud del coseno (si `num` es 1) o la distancia euclídea (si `num` es 2) para determinar la categoría más similar.

    Args:
        query_codificado (list of float): El vector codificado de la consulta.
        diccionario_corpus_codificacion (dict): Diccionario donde las claves son las categorías y los valores son listas de vectores codificados del corpus.
        num (int): Determina el tipo de similitud a usar:
            - 1 para similitud del coseno.
            - 2 para distancia euclídea.

    Returns:
        str: La categoría del corpus más similar a la consulta según la similitud calculada.

    Notes:
        - Si `num` es 1, se utiliza la similitud del coseno para determinar la categoría más similar.
        - Si `num` es 2, se utiliza la distancia euclídea para determinar la categoría más similar.
    """
    categoria_query = similitud(query_codificado, diccionario_corpus_codificacion, num)
    return categoria_query


In [ ]:
for clave,query_codificado in diccionario_querys_codificacion.items():
  print(f"Usando similitud del coseno, Query {clave} pertence a {clasificar(query_codificado,diccionario_corpus_codificacion,1)}")

Usando similitud del coseno, Query 0 pertence a deporte
Usando similitud del coseno, Query 1 pertence a politica
Usando similitud del coseno, Query 2 pertence a entretenimiento
Usando similitud del coseno, Query 3 pertence a entretenimiento
Usando similitud del coseno, Query 4 pertence a entretenimiento
Usando similitud del coseno, Query 5 pertence a deporte
Usando similitud del coseno, Query 6 pertence a politica
Usando similitud del coseno, Query 7 pertence a deporte


In [ ]:
for clave,query_codificado in diccionario_querys_codificacion.items():
  print(f"Usando similitud euclidea, Query {clave} pertence a {clasificar(query_codificado,diccionario_corpus_codificacion,2)}")

Usando similitud euclidea, Query 0 pertence a deporte
Usando similitud euclidea, Query 1 pertence a politica
Usando similitud euclidea, Query 2 pertence a entretenimiento
Usando similitud euclidea, Query 3 pertence a entretenimiento
Usando similitud euclidea, Query 4 pertence a politica
Usando similitud euclidea, Query 5 pertence a deporte
Usando similitud euclidea, Query 6 pertence a politica
Usando similitud euclidea, Query 7 pertence a politica


# Reporte

## 1. Resultados

Usar tablas y gráficos para explicar los resultados obtenidos.

| **Modelo** | **Función de Similitud** | **Accuracy** (Tasa de acierto) |
|---|---|---|
| 01 | Coseno | 100% |
| 02 | Euclídea | 75% |


### Modelos

*   01: Similitud del coseno. Explicación\
Similitud del Coseno y TF-IDF: El concepto básico detrás de la similitud del coseno es medir la similitud entre dos vectores en un espacio multidimensional. Cada documento del corpus y cada query (consulta) se representan como vectores en un espacio vectorial, donde cada dimensión representa una palabra del diccionario. Estos vectores están codificados utilizando la técnica TF-IDF (Term Frequency-Inverse Document Frequency), que asigna pesos a las palabras basándose en su frecuencia en el documento y en cuántos documentos la contienen.

* Cálculo de la similitud del coseno con TF-IDF:\
Se define la función coseno que toma dos argumentos: query_codificado y corpus_codificado, que representan los vectores de características de la consulta y el corpus, respectivamente.\
Se inicializan tres variables (sumatoria_a_cuadrado, sumatoria_b_cuadrado, sumatoria_a_por_b) para calcular diferentes partes de la fórmula de similitud coseno.\
Se itera sobre los elementos del vector corpus_codificado y calcula la suma de los cuadrados de estos elementos, almacenando el resultado en sumatoria_a_cuadrado.\
Se itera sobre los elementos del vector query_codificado y calcula la suma de los cuadrados de estos elementos, almacenando el resultado en sumatoria_b_cuadrado.\
Se utiliza la función zip para iterar simultáneamente sobre los elementos de corpus_codificado y query_codificado. Calcula el producto punto entre los elementos correspondientes de ambos vectores y acumula el resultado en sumatoria_a_por_b.\
Se comprueba si alguno de los vectores tiene una longitud nula (es decir, si alguno de los vectores es un vector nulo) para evitar divisiones por cero en el cálculo de la similitud coseno.\
Se calcula la similitud coseno dividiendo la suma acumulada de productos punto (sumatoria_a_por_b) entre la raíz cuadrada del producto de las sumas de cuadrados de los dos vectores.



*   02: Similitud Euclídea. Explicación\
Función Euclidiana: La función euclides calcula la similitud euclidiana entre el vector del query y un vector del corpus codificado en TF-IDF. La similitud euclidiana mide la distancia entre dos puntos en un espacio n-dimensional utilizando la fórmula de la distancia euclidiana.

* Cálculo de la Similitud Euclidiana:\
Se define la función euclides que toma dos argumentos: query_codificado y corpus_codificado, que representan los vectores de características del query y el corpus, respectivamente.\
Se inicializa una variable sumatoria_b_menos_a_cuadrado para acumular la suma de los cuadrados de las diferencias entre los elementos correspondientes de query_codificado y corpus_codificado.\
Se utiliza la función zip para iterar simultáneamente sobre los elementos de corpus_codificado y query_codificado. Para cada par de elementos, calcula la diferencia entre los elementos correspondientes, eleva al cuadrado esta diferencia y acumula el resultado en sumatoria_b_menos_a_cuadrado.\
Finalmente, se calcula la raíz cuadrada de la suma acumulada (sumatoria_b_menos_a_cuadrado) para obtener la distancia euclidiana.